# Step 0. Parameter Setting

In [4]:
import dtnmonitor_v2 as dtn
import os,sys


### P1. SELECT THE MONITOR NETWORK INTERFACE
## used in Step 1,2,3 , if you don't know the interface name, you can use 'all'
## make sure the interface is correct, error_naming or without network flow will get empty grath
#dtn.interface = 'eth0.1301'
#dtn.interface = 'eth0.2038' 
dtn.interface = 'ens1f0.3061'
#dtn.interface = 'eth1'
#dtn.interface = 'all'


### P2. SELECT THE INPUT FILE , 
## used in Step 1 : file IO read file



file_read_nuttcp=[
"/data/disk0/sc17/100gtest",
"/data/disk1/sc17/100gtest",
"/data/disk2/sc17/100gtest",
"/data/disk3/sc17/100gtest",
"/data/disk4/sc17/100gtest",
"/data/disk5/sc17/100gtest",
"/data/disk6/sc17/100gtest",
"/data/disk7/sc17/100gtest",
    # "/foo/bar/filennn"
     ]

file2_read_nuttcp=[
"/data/disk0/sc17/fftest",
"/data/disk1/sc17/fftest",
"/data/disk2/sc17/fftest",
"/data/disk3/sc17/fftest",
"/data/disk4/sc17/fftest",
"/data/disk5/sc17/fftest",
"/data/disk6/sc17/fftest",
"/data/disk7/sc17/fftest",
    # "/foo/bar/filennn"
     ]






### P3. NUTTCP SERVER IP 
## used in Step 2,3 : input nuttcp server's ip
nuttcp_server="192.168.61.57"

### P4. NUTTCP THREADS
## used in Step 2,3 :input threads number
count = 8

## run fio and check status
### select graph_mode [0|1] 
#graph_mode=0  # all, 
#graph_mode=1  # 100pt
graph_mode=0  

### Show Disk and Interface
import netifaces
import os
list=netifaces.interfaces()
print("==========================================================")
print('{:40s} {:20s} '.format("Network Interface","IP"))
print("==========================================================")
for inf in list:
    #print(inf)
    ff=netifaces.ifaddresses(inf)
    #print(ff)
    try:
        addr=ff[netifaces.AF_INET][0]['addr']
    except:
        continue
    print('{:40s} {:20s} '.format(inf,addr)) 
import subprocess
import re
df = subprocess.Popen(["df","-h"], stdout=subprocess.PIPE)
output = df.communicate()[0]
dfs = output.decode("utf8").split("\n")
dfs.pop(0)
print("==========================================================")
print('{:35s} {:7s}  {:7s} {:7s}'.\
      format("Disk mountpoint","size","avail","used%"))
print("==========================================================")
for i in dfs:
    try:
        xx = re.sub(" +"," ",i).split(" ")
        if len(xx) == 6:
            device, size, used, available, percent, mountpoint = xx
            print('{:35s} {:7s}  {:7s} {:7s}'\
                  .format(mountpoint,size,available,percent))
    except:
        continue

def exec_print(command):
    process = subprocess.Popen([command], stdout=subprocess.PIPE,stderr=subprocess.STDOUT, shell=True)
    for line in iter(process.stdout.readline, b''):
        # system print
        sys.stdout.write(line)   

Network Interface                        IP                   
lo                                       127.0.0.1            
enp20s0                                  165.124.33.143       
ens1f0.3061                              192.168.61.50        
ens1f0.2038                              10.250.38.50         
ens4.1301                                10.10.14.251         
Disk mountpoint                     size     avail   used%  
/dev                                32G      32G     0%     
/dev/shm                            32G      32G     0%     
/run                                32G      32G     1%     
/sys/fs/cgroup                      32G      32G     0%     
/                                   50G      21G     60%    
/boot                               1014M    790M    23%    
/home                               151G     108G    29%    
/run/user/2003                      6.3G     6.3G    0%     
/data/disk0                         373G     155G    59%    
/data/disk1 

# Step 1. Test file IO

# setup fio parameters
# first, filename is assigned with existed and >1GB file path 
randrepeat=" --randrepeat=0" #--randrepeat=0 
bs=" --bs=1M" #--bs=1M 
ioengine=" --ioengine=sync "#--ioengine=sync 
tuntime=" --runtime=30" #--runtime=30 
iodepth=" --iodepth=32"#--iodepth=32 
name=" --name=drive0"#--name=drive0 
size=" --size=100G "#--size=100G 
filename=" --filename="+file_read_fio #--filename=filename
numjobs=" --numjobs="+str(count) 



## method 1 : launching subprocess
#for i in range(count):
#    filename=" --filename="+file_read_fio #--filename=filename
#    cmd="fio --thread --rw=read --readonly --norandommap --group_reporting --time_based "
#    command = cmd + randrepeat + bs + ioengine + tuntime + iodepth + name + size + filename +" & "
#    print(command)
#    os.system(command)

## method 2 : multithreads by parameters 
filename=" --filename="+file_read_fio #--filename=filename
cmd="sleep 3; fio --thread --rw=read --readonly --norandommap --group_reporting --time_based"
#command = cmd + randrepeat + bs + ioengine + tuntime + iodepth + name + size + numjobs + filename 
command = cmd + randrepeat + bs + ioengine + tuntime + iodepth + name + size + filename 
print(command)


os.system(command)

dtn.exec_command("sleep 30" ,graph_mode)


# Step 2. Test network transfer by memory

command = ""
for i in range(count):
     command += "nuttcp -t -i1 -xc "+str(i)+" -w 110M -T 30s -P 5200"+str(i)+" -p 5210"+str(i)+ " "+ nuttcp_server +" & "
#    if i != (count-1):
#        command += "nuttcp -t -i1 -xc "+str(i)+" -w 110M -T 30s -P 5200"+str(i)+" -p 5210"+str(i)+ " "+ nuttcp_server +" & "
#    else: 
#        command += "nuttcp -t -i1 -xc "+str(i)+" -w 110M -T 30s -P 5200"+str(i)+" -p 5210"+str(i)+ " "+ nuttcp_server +" & "
        
#print(command)
os.system(command)
dtn.exec_command("sleep 40",graph_mode)



# Step 3. Test network transfer by disk

In [7]:
for i in range(count):
    command = "nuttcp -t -i1 -sdz -xc "+str(i)+" -w 110M -P 5300"+str(i)+" -p 5310"+str(i)+ " "+ nuttcp_server +" < "+ file_read_nuttcp[i] +" & "
    print(command)
    #os.system(command)
    command = "nuttcp -t -i1 -sdz -xc "+str(i)+" -w 110M -P 5320"+str(i)+" -p 5330"+str(i)+ " "+ nuttcp_server +" < "+ file2_read_nuttcp[i] +" & "
    print(command)
    #os.system(command)
 


#dtn.exec_command("sleep 300",graph_mode)

nuttcp -t -i1 -sdz -xc 0 -w 110M -P 53000 -p 53100 192.168.61.57 < /data/disk0/sc17/100gtest & 
nuttcp -t -i1 -sdz -xc 0 -w 110M -P 53200 -p 53300 192.168.61.57 < /data/disk0/sc17/fftest & 
nuttcp -t -i1 -sdz -xc 1 -w 110M -P 53001 -p 53101 192.168.61.57 < /data/disk1/sc17/100gtest & 
nuttcp -t -i1 -sdz -xc 1 -w 110M -P 53201 -p 53301 192.168.61.57 < /data/disk1/sc17/fftest & 
nuttcp -t -i1 -sdz -xc 2 -w 110M -P 53002 -p 53102 192.168.61.57 < /data/disk2/sc17/100gtest & 
nuttcp -t -i1 -sdz -xc 2 -w 110M -P 53202 -p 53302 192.168.61.57 < /data/disk2/sc17/fftest & 
nuttcp -t -i1 -sdz -xc 3 -w 110M -P 53003 -p 53103 192.168.61.57 < /data/disk3/sc17/100gtest & 
nuttcp -t -i1 -sdz -xc 3 -w 110M -P 53203 -p 53303 192.168.61.57 < /data/disk3/sc17/fftest & 
nuttcp -t -i1 -sdz -xc 4 -w 110M -P 53004 -p 53104 192.168.61.57 < /data/disk4/sc17/100gtest & 
nuttcp -t -i1 -sdz -xc 4 -w 110M -P 53204 -p 53304 192.168.61.57 < /data/disk4/sc17/fftest & 
nuttcp -t -i1 -sdz -xc 5 -w 110M -P 53005 -p 53105

In [6]:
for i in range(count):
    command = "nuttcp -t -i1 -sdz -xc "+str(i)+" -w 110M -P 5300"+str(i)+" -p 5310"+str(i)+ " "+ nuttcp_server +" < "+ file_read_nuttcp[i] +" & "
    #print(command)
    os.system(command)
    command = "nuttcp -t -i1 -sdz -xc "+str(i)+" -w 110M -P 5320"+str(i)+" -p 5330"+str(i)+ " "+ nuttcp_server +" < "+ file2_read_nuttcp[i] +" & "
    os.system(command)
 


dtn.exec_command("sleep 300",graph_mode)

Exception ignored in: <bound method Socket.__del__ of <zmq.sugar.socket.Socket object at 0x7f2740e27ee8>>
Traceback (most recent call last):
  File "/usr/lib64/python3.4/site-packages/zmq/sugar/socket.py", line 69, in __del__
    if not self._shadow:
  File "/usr/lib64/python3.4/site-packages/zmq/sugar/socket.py", line 69, in __del__
    if not self._shadow:
  File "/home/sc17/DTN_monitor/dtnmonitor_v2.py", line 104, in localtrace
    raise SystemExit()
SystemExit: 
